In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
from skimage import io
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

In [ ]:
def filter_masks_by_size(masks, image, max_size_factor=0.1):
    """
    Filters masks based on their size to remove very large masks dynamically.
    Small masks are not filtered out.
    """
    filtered_masks = []
    total_pixels = image.shape[0] * image.shape[1]
    max_size = max_size_factor * total_pixels

    for mask in masks:
        mask_size = np.sum(mask['segmentation'])
        if mask_size <= max_size:
            filtered_masks.append(mask)
    return filtered_masks


def calculate_circularity(mask):
    """
    Calculate the circularity of a given mask.
    Circularity is defined as 4 * pi * (area / perimeter^2).
    """
    area = np.sum(mask)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        perimeter = cv2.arcLength(contours[0], True)
        if perimeter == 0:
            return 0
        circularity = 4 * np.pi * (area / (perimeter ** 2))
    else:
        circularity = 0
    return circularity

def average_intensity(image, mask):
    """
    Calculate the average intensity of the masked area in the image.
    """
    masked_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8))
    return np.mean(masked_image[mask > 0])


def calculate_iou(mask1, mask2):
    """
    Calculate the Intersection over Union (IoU) of two masks.
    """
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou


def apply_kmeans(masks, image):
    """
    Apply K-means clustering to split masks into darker and lighter clusters based on average intensity.
    """
    avg_intensities = [average_intensity(image, mask['segmentation']) for mask in masks]
    features = np.array(avg_intensities).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(features)
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    darker_cluster_index = 0 if cluster_centers[0] < cluster_centers[1] else 1
    darker_masks = [masks[i] for i in range(len(masks)) if labels[i] == darker_cluster_index]
    lighter_masks = [masks[i] for i in range(len(masks)) if labels[i] != darker_cluster_index]
    return darker_masks, lighter_masks


def show_anns(anns):
    """
    Display annotations on the image.
    """
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

def show_masks(image, masks, title, output_path):
    """
    Save and display the masks on the image.
    """
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(image)
    show_anns(masks)
    plt.title(title)
    ax.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def resolve_overlaps(masks, iou_threshold=0.5):
    """
    Resolve overlaps between masks, keeping the mask with the larger area in case of overlap.
    """
    keep_masks = []
    n = len(masks)
    removed = set()
    masks.sort(key=lambda x: np.sum(x['segmentation']), reverse=True)

    for i in range(n):
        if i in removed:
            continue
        current_mask = masks[i]['segmentation']
        for j in range(i + 1, n):
            if j in removed:
                continue
            compare_mask = masks[j]['segmentation']
            if calculate_iou(current_mask, compare_mask) > iou_threshold:
                removed.add(j)

        keep_masks.append(masks[i])

    return keep_masks

def calculate_coverage_percentage(image, masks):
    """
    Calculate the coverage percentage of the masks on the image.
    """
    total_pixels = image.shape[0] * image.shape[1]
    mask_area_sum = sum(np.sum(mask['segmentation']) for mask in masks)
    coverage_percentage = (mask_area_sum / total_pixels) * 100
    return coverage_percentage

In [ ]:
def process_directory_holes(image_directory, output_directory, coverage_csv):
    """
    Process images in the directory, generating masks and calculating properties including coverage percentage.
    """
    device = torch.device("cuda")
    sam_checkpoint = "sam_vit_l_0b3195.pth"
    model_type = "vit_l"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam)
    mask_generator_2 = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=52,
        pred_iou_thresh=0.7,
        stability_score_thresh=0.92,
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=60
    )
    coverage_data = []

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(image_directory):
        if filename.lower().endswith(".tif"):
            file_path = os.path.join(image_directory, filename)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            masks = mask_generator.generate(image)
            masks2 = mask_generator_2.generate(image)

            darker_masks1, lighter_masks1 = apply_kmeans(masks, image)
            darker_masks2, lighter_masks2 = apply_kmeans(masks2, image)

            # Combine, resolve overlaps, and filter for darker masks
            combined_darker_masks = darker_masks1 + darker_masks2
            combined_darker_masks = resolve_overlaps(combined_darker_masks)
            max_size_factor = 0.1
            filtered_darker_masks = filter_masks_by_size(combined_darker_masks, image, max_size_factor)
            output_path = os.path.join(output_directory, f"{filename[:-4]}_darker.jpg")
            show_masks(image, filtered_darker_masks, "Darker Masks", output_path)
            coverage_percentage = calculate_coverage_percentage(image, filtered_darker_masks)

            # Combine, resolve overlaps, and filter for lighter masks
            combined_lighter_masks = lighter_masks1 + lighter_masks2
            combined_lighter_masks = resolve_overlaps(combined_lighter_masks)
            filtered_lighter_masks = filter_masks_by_size(combined_lighter_masks, image, max_size_factor)
            output_path = os.path.join(output_directory, f"{filename[:-4]}_lighter.jpg")
            show_masks(image, filtered_lighter_masks, "Lighter Masks", output_path)

            # Calculate average size and circularity for lighter masks
            average_size = np.mean([np.sum(mask['segmentation']) for mask in filtered_lighter_masks])
            average_circularity = np.mean([calculate_circularity(mask['segmentation']) for mask in filtered_lighter_masks])
            coverage_data.append([filename, coverage_percentage, average_size, average_circularity])

    df = pd.DataFrame(coverage_data, columns=['Filename', 'Coverage Percentage', 'Average Size', 'Average Circularity'])
    df.to_csv(coverage_csv, index=False)

In [ ]:
def process_directory_no_holes(image_directory, output_directory, coverage_csv):
    """
    Process images in the directory, generating masks and calculating average size and circularity.
    """
    device = torch.device("cuda")
    sam_checkpoint = "sam_vit_l_0b3195.pth"
    model_type = "vit_l"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam)
    mask_generator_2 = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=52,
        pred_iou_thresh=0.7,
        stability_score_thresh=0.92,
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=60
    )
    coverage_data = []

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(image_directory):
        if filename.lower().endswith(".tif"):
            file_path = os.path.join(image_directory, filename)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            masks = mask_generator.generate(image)
            masks2 = mask_generator_2.generate(image)

            # Combine and resolve overlaps for all masks
            combined_masks = masks + masks2
            non_overlapping_masks = resolve_overlaps(combined_masks)

            max_size_factor = 0.1
            filtered_masks = filter_masks_by_size(non_overlapping_masks, image, max_size_factor)
            output_path = os.path.join(output_directory, f"{filename[:-4]}_masks.jpg")
            show_masks(image, filtered_masks, "Masks", output_path)

            # Calculate average size and circularity for masks
            average_size = np.mean([np.sum(mask['segmentation']) for mask in filtered_masks])
            average_circularity = np.mean([calculate_circularity(mask['segmentation']) for mask in filtered_masks])
            coverage_data.append([filename, average_size, average_circularity])

    df = pd.DataFrame(coverage_data, columns=['Filename', 'Average Size', 'Average Circularity'])
    df.to_csv(coverage_csv, index=False)

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Set your directories and CSV file path
image_directory_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/HOLES'
image_directory_no_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/NO HOLES'
output_directory_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/Processed_Images_holes'
output_directory_no_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/Processed_Images_no_holes'

info_csv_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/Processed_Images_holes.csv'
info_csv_no_holes = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/Processed_Images_no_holes.csv'

# Execute processing
process_directory_holes(image_directory_holes, output_directory_holes, info_csv_holes)
process_directory_no_holes(image_directory_no_holes, output_directory_no_holes, info_csv_no_holes)
